In [1]:
import pandas as pd
import numpy as np
from numpy import array
from keras import layers
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Embedding,Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from nltk.tokenize import word_tokenize
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape

In [2]:
data = pd.read_csv(r"dataset.csv")

In [3]:
print(data)

                                                     url  label
0      http://007indien.blogspot.com/2013/08/daftar-l...      0
1                              http://021jiaju.com/xhsd/      0
2         http://03.rospotrebnadzor.ru/content/336/9833/      0
3               http://039.holidayblog.jp/?cat=7&paged=2      0
4      http://03online.com/news/boli_vnizu_zhivota_za...      0
...                                                  ...    ...
20123  http://click.partner.roamans.com/c6/7310437653...      1
20124  https://click.securedvisit.com/c6/7310437653_6...      1
20125  http://click.partner.roamans.com/c6/7310437653...      1
20126  https://www.roamans.com/?affiliate_id=054&affi...      1
20127  https://click.securedvisit.com/c6/7310437653_6...      1

[20128 rows x 2 columns]


In [4]:
urls = data.iloc[:,0]

In [5]:
print(urls)

0        http://007indien.blogspot.com/2013/08/daftar-l...
1                                http://021jiaju.com/xhsd/
2           http://03.rospotrebnadzor.ru/content/336/9833/
3                 http://039.holidayblog.jp/?cat=7&paged=2
4        http://03online.com/news/boli_vnizu_zhivota_za...
                               ...                        
20123    http://click.partner.roamans.com/c6/7310437653...
20124    https://click.securedvisit.com/c6/7310437653_6...
20125    http://click.partner.roamans.com/c6/7310437653...
20126    https://www.roamans.com/?affiliate_id=054&affi...
20127    https://click.securedvisit.com/c6/7310437653_6...
Name: url, Length: 20128, dtype: object


In [6]:
labels = data.iloc[:,1]
print(labels)

0        0
1        0
2        0
3        0
4        0
        ..
20123    1
20124    1
20125    1
20126    1
20127    1
Name: label, Length: 20128, dtype: int64


In [7]:
vocab_size = 70
encoded_reviews = [one_hot(url,vocab_size) for url in urls]
print(f'encoded reviews: {encoded_reviews}')

encoded reviews: [[38, 16, 6, 61, 15, 48, 64, 18, 37, 65, 64, 10], [38, 8, 61, 38], [38, 11, 19, 7, 52, 29, 13], [38, 38, 26, 50, 51, 46, 4, 26], [38, 57, 61, 19, 18, 55, 54, 69, 60, 52, 57, 18, 32], [38, 57, 61, 19, 58, 24, 45, 5, 63, 21, 52, 5, 12, 66], [38, 64, 43, 61, 32, 30], [38, 52, 20, 7, 61, 30, 33, 67, 10], [38, 23, 61, 23, 55, 22, 28, 30], [38, 28, 61, 22, 2, 22, 28, 62], [38, 9, 61, 50, 58, 10], [38, 36, 61, 60, 19, 10], [38, 35, 35, 6, 61, 24, 56, 33, 45, 28, 45, 10], [38, 25, 24, 61, 19, 25, 65, 11, 65], [38, 41, 2, 47, 20, 35, 8], [38, 41, 2, 47, 20, 35, 68], [38, 64, 13, 50, 24, 33, 1], [38, 24, 26, 27, 12, 4, 33, 26], [38, 24, 10, 47, 61, 37, 39], [38, 30, 27, 53, 33, 10], [38, 30, 42, 52, 41, 32, 52, 59, 65], [38, 3, 57, 58, 39, 11, 8, 53, 9, 46, 34, 65], [38, 57, 52, 61, 52, 42, 17, 34, 4, 35, 20, 10], [38, 20, 7, 40, 59, 37, 18, 68, 16], [38, 48, 7, 66, 36, 9], [38, 10, 61, 13, 8, 10], [38, 34, 7, 14, 65], [38, 50, 7, 18, 53, 68, 26, 65], [38, 53, 24, 38, 28, 47, 43

In [8]:
max_length = 256
padded_reviews = pad_sequences(encoded_reviews,maxlen=max_length,padding='post')
print(padded_reviews[0].shape)

(256,)


In [9]:
model = Sequential()
embedding_layer = Embedding(input_dim=vocab_size,output_dim=16,input_length=256)
model.add(embedding_layer)
model.add(Reshape((128, 16), input_shape=(256,70)))
model.add(layers.Conv2D(8, (65, 16), activation="relu"))
model.add(layers.MaxPooling2D((2,2), padding='same'))
model.add(layers.Conv2D(16, (17,1), activation="relu"))
model.add(layers.MaxPooling2D((2,2), padding='same'))
model.add(layers.Conv2D(32, (1,1), activation="relu"))
model.add(layers.MaxPooling2D((8,8), padding='same'))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation="relu"))
model.add(layers.Dense(1,  activation="sigmoid"))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
print(model.summary())

ValueError: total size of new array must be unchanged, input_shape = [256, 16], output_shape = [128, 16]

In [ ]:
model.fit(padded_reviews,labels,batch_size=32,epochs=100)

In [ ]:
loss, acc = model.evaluate(padded_reviews, labels, verbose=0)
print(acc)